In [ ]:
import os
import myServices as ms
import randForest as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
sklearn.metrics.get_scorer_names()

In [8]:
print(np.linspace(50,110, 33).astype(int))

[ 50  51  53  55  57  59  61  63  65  66  68  70  72  74  76  78  80  81
  83  85  87  89  91  93  95  96  98 100 102 104 106 108 110]


In [ ]:
DS = pd.read_csv('basin1CleanToTrain.csv', index_col = None)

In [ ]:
print(DS.keys())
DS.info()
print(DS.keys())


In [ ]:
import 

In [ ]:
s = {}
m = {'cv': 4, 'error_score': 'nan', 'estimator__bootstrap': True, 
     'estimator__ccp_alpha': 0.0, 'estimator__class_weight': None, 
     'estimator__criterion': 'entropy', 'estimator__max_depth': None, 
     'estimator__max_features': 'sqrt', 'estimator__max_leaf_nodes': None, 
     'estimator__max_samples': None, 'estimator__min_impurity_decrease': 0.0, 
     'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 
     'estimator__min_weight_fraction_leaf': 0.0, 'estimator__n_estimators': 100,
     'estimator__n_jobs': None, 'estimator__oob_score': False, 'estimator__random_state': 50,
     'estimator__verbose': 0, 'estimator__warm_start': False, 
     'estimator': 'RandomForestClassifier(criterion =entropy, random_state=50)', 
     'n_jobs': -1, 'param_grid': {'n_estimators': [8], 'max_depth': [None], 'max_features': [0.8], 'max_leaf_nodes': [None], 
                                  'bootstrap': [False]}, 'pre_dispatch': '2*n_jobs', 
     'refit': True, 'return_train_score': False, 'scoring': 'roc_auc', 'verbose': 5}

In [ ]:
s['test'] = m

In [ ]:
print(s)